<a href="https://colab.research.google.com/github/fportoDexl/cursoBigData/blob/main/src/Transforma%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://drive.google.com/uc?id=1Oj5GoldtG5ZWgGOyRLGkhthLR2nc6Lcj'>

# Exemplo 4 Aula 2 - Transformações

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 50.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=4a3269774d2738f94c9cdac37e0949088a1ca24905cb0c9f2ddd653d8e509785
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark import SparkContext
import random

In [3]:
spark = SparkContext(appName="Transform",master="local")

# Map

In [4]:
def find_k_nearest_neighbors(k,p, points):
    distances = [abs(p-point) for point in points]
    neighbors = distances[0:k]
    return [p,neighbors]


In [5]:
number_rdd = spark.parallelize(range(1,1000))
points =  random.sample(range(1,5000),1000)  
print (len(points))  

1000


In [6]:
neighborsRDD = number_rdd.map (lambda p: find_k_nearest_neighbors(3,p, points))
neighborsRDD.take(10)

[[1, [1066, 2564, 1957]],
 [2, [1065, 2563, 1956]],
 [3, [1064, 2562, 1955]],
 [4, [1063, 2561, 1954]],
 [5, [1062, 2560, 1953]],
 [6, [1061, 2559, 1952]],
 [7, [1060, 2558, 1951]],
 [8, [1059, 2557, 1950]],
 [9, [1058, 2556, 1949]],
 [10, [1057, 2555, 1948]]]

# FlatMap

In [7]:
inRDD = spark.parallelize([2,3,4])

In [8]:
inRDD.count()

3

In [9]:
inRDD.flatMap(lambda x:range(1,x)).collect()

[1, 1, 2, 1, 2, 3]

In [10]:
!wget https://raw.githubusercontent.com/fportoDexl/cursoBigData/main/data/word-count.txt

--2022-04-14 14:43:49--  https://raw.githubusercontent.com/fportoDexl/cursoBigData/main/data/word-count.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3875 (3.8K) [text/plain]
Saving to: ‘word-count.txt’

word-count.txt      100%[===================>]   3.78K  --.-KB/s    in 0s      

2022-04-14 14:43:50 (45.1 MB/s) - ‘word-count.txt’ saved [3875/3875]



In [11]:
words=spark.textFile("word-count.txt")
wordMapRDD = words.map(lambda w: w.split(" "))
totalMap=wordMapRDD.count()
wordsFlatMapRDD = words.flatMap(lambda w: w.split(" "))
totalflatMap=wordsFlatMapRDD.count()
print("Total Map: "+str(totalMap)+" TotalFlatMap: "+str(totalflatMap))

Total Map: 38 TotalFlatMap: 604


# FlatMapValues

In [13]:
xRDD = spark.parallelize([("a", ["x", "y", "z"]), ("b", ["p", "r"])])

def mirror (x): return x

xRDD.flatMapValues(mirror).collect()



[('a', 'x'), ('a', 'y'), ('a', 'z'), ('b', 'p'), ('b', 'r')]

# MapPartitions

In [14]:
from random import sample

In [16]:
def processPartition(part):
    for element in part:
      yield element

In [17]:
sampleRDD = spark.parallelize([1,2,3,4,5,6,7],2)
resultRDD = sampleRDD.mapPartitions(processPartition)
resultRDD.take(7)


[1, 2, 3, 4, 5, 6, 7]

# PairRDD

In [18]:
data = [('Project', 1), ('Gutenberg’s', 1), ('Alice’s', 1),('Adventures', 1),('in', 1),
('Wonderland', 1),('Project', 1),('Gutenberg’s', 1),('Adventures', 1),('in', 1),
('Wonderland', 1),('Project', 1),('Gutenberg’s', 1)]
pairrdd=spark.parallelize(data)
pairrdd.coalesce(1).saveAsTextFile("/data/")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#ReduceByKey

In [72]:
reducedbykeyRDD = pairrdd.reduceByKey(lambda a,b:a+b)
reducedbykeyRDD.collect()

[('Project', 3),
 ('Gutenberg’s', 3),
 ('Alice’s', 1),
 ('Adventures', 2),
 ('in', 2),
 ('Wonderland', 2)]